## Weight & Biases
Zur Nachverfolgung und Analyse der Model-Performance

### Dataset
- **Train:** Modell wird trainiert & validiert. Modell & Pipeline serialisiert
- **Test:** Serialisierte Pipeline & Modell wird auf Test-Set angewendet

In [1]:
train_run = True
residuals_plot = False

### Initialisieren

In [2]:
from typing import Final

import wandb
import pandas as pd
from joblib import load
from numpy import mean
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, max_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

import src.package.consts as c
import src.package.importer as im
import src.package.importer_usages as imp_usg
import src.package.ml_helper as ml_helper
import src.package.shared as sh

pd.set_option('display.max_rows', 500)  # to show all value_counts

MODEL_GRADIENT_BOOSTING: Final = 'GradientBoosting'
MODEL_LINEAR_REGRESSION: Final = 'LinearRegression'
current_model = MODEL_GRADIENT_BOOSTING

TAG_TRAIN_SET: Final = 'train-set'
TAG_TEST_SET: Final = 'test-set'


def load_dataset(residual_test_run: bool = False):
    set_file = 'train_set.csv' if train_run else 'test_set.csv'
    df = im.get_extended_dataset(f'../package/datasets/{set_file}')
    df, usage_types = imp_usg.extract_usage_details(df, highest_only=True, combine_garages=True)

    additional_features = [c.FIELD_AREA_TOTAL_FLOOR_416]

    if train_run and not residual_test_run:
        df = ml_helper.remove_outliers(df, factor=2.3)
        return ml_helper.ml_dataset_full(df, field_to_predict=c.FIELD_AREA_USAGE,
                                         additional_features=additional_features)
    else:
        # load transform pipeline for test_set
        pipeline = load(sh.export_directory_path(f'fitted_pipeline.joblib'))
        return ml_helper.ml_dataset_full(df, field_to_predict=c.FIELD_AREA_MAIN_USAGE,
                                         additional_features=additional_features,
                                         fitted_pipeline=pipeline)


def init_wandb(X, tag_set, model, related_run=None):
    wandb.login()

    config = {
        'model': model,
        'features': ', '.join(X.columns.values),
        'dataset-count': len(X.index),
        'description': 'New Dataset, Impute volume 416 (clustered), Removed outliers (2.3), Drop all, OneHotEncoding, Default Hyperparameter'
    }

    if related_run is not None:
        config['related_run'] = related_run

    print(config)
    wandb.init(project='Metriken Bauwesen', entity='devcore', config=config, tags=[tag_set])

### Train-Run

In [3]:
if train_run:
    X, y = load_dataset()
    init_wandb(X, TAG_TRAIN_SET, current_model)

    regr = GradientBoostingRegressor(random_state=0,
                                     learning_rate=0.0134,
                                     subsample=0.835,
                                     n_estimators=768,
                                     max_depth=8,
                                     min_samples_split=10,
                                     loss='lad')

    scores_map = ml_helper.cross_validation(regr, X, y)

    # log result of cross validation to wandb
    if not residuals_plot:
        for key in scores_map.keys():
            if key in {'fit_time', 'score_time'}:
                scores = scores_map[key]
                # wandb.log({f'{key}_mean': mean(scores)})
                print({f'{key}_mean': mean(scores)})

    # serialize model for test set
    regr.fit(X, y)
    sh.serialize_object(regr, 'fitted_model')

    # sync residual plot
    if residuals_plot:
        X_test, y_test = load_dataset(residual_test_run=True)
        wandb.sklearn.plot_regressor(regr, X, X_test, y, y_test)


Location: /Users/lucadietiker/Documents/05_Projects/Python.nosync/ip5-metriken-bauwesen/export/cluster_means.joblib
Location: /Users/lucadietiker/Documents/05_Projects/Python.nosync/ip5-metriken-bauwesen/export/fitted_pipeline.joblib


wandb: Currently logged in as: devcore (use `wandb login --relogin` to force relogin)


{'model': 'GradientBoosting', 'features': 'num_floors_underground, num_floors_overground, garage_combined, total_expenses, primary_percentage, volume_total_416, area_total_floor_416, x0_ANDERES, x0_BEHERBERGUNG, x0_BETRIEB, x0_BUERO, x0_GESUNDHEIT, x0_HALLEN, x0_KULTUS_KULTUR, x0_OFFENE_BAUTEN, x0_SCHULEN, x0_TECHNIK, x0_WOHNEN', 'dataset-count': 209, 'description': 'New Dataset, Impute volume 416 (clustered), Removed outliers (2.3), Drop all, OneHotEncoding, Default Hyperparameter'}


wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Location: /Users/lucadietiker/Documents/05_Projects/Python.nosync/ip5-metriken-bauwesen/export/fitted_model.joblib


wandb: 
wandb: Plotting Regressor.
wandb: Logged summary metrics.
wandb: Logged learning curve.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
wandb: Logged outlier candidates.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged residuals.


### Test-Run

In [4]:
if not train_run:
    X, y = load_dataset()
    # init_wandb(X, TAG_TEST_SET, current_model, related_run='peach-universe-66')

    regr = load(sh.export_directory_path(f'fitted_model.joblib'))
    predictions = regr.predict(X)

    r2_score = regr.score(X, y)
    print("R^2 value: ", r2_score)
    #wandb.log({'test_r2_mean': r2_score})

    neg_mean_squared_error = -mean_squared_error(y, predictions, squared=False)
    print("neg_mean_squared_error", neg_mean_squared_error)
    #wandb.log({'test_neg_root_mean_squared_error_mean': neg_mean_squared_error})

    neg_mean_absolute_error = -mean_absolute_error(y, predictions)
    print("neg_mean_absolute_error", neg_mean_absolute_error)
    #wandb.log({'test_neg_mean_absolute_error_mean': neg_mean_absolute_error})

    neg_mape = -mean_absolute_percentage_error(y, predictions)
    print("neg_mean_absolute_percentage_error", neg_mape)
    #wandb.log({'test_neg_mean_absolute_percentage_error_mean': neg_mape})

    neg_max_error = -max_error(y, predictions)
    print("neg_max_error", neg_max_error)
    #wandb.log({'test_max_error_mean': neg_max_error})

    #wandb_plot_regressor(regr, X, y)

# charts.plot_feature_importance(regr.feature_importances_, X.columns, 'GRADIENT BOOSTING')